# Exercises 1 - Representing Adversarial Game States

I refactored the code substantially to be more flexible.

In particular, I separated the game engine from the game state, so that I could
inject different games (e.g. an even-or-odd style game) or different engines (the random move picker vs minimax).

So the code below represents that.

Components:
- Game package
  - Game - an abstract class that has a bunch of methods any game has to implement
  - TicTacToe - an implementation of Game for TicTacToe
  - Count - another implementation of Game that's a little easier to visualize and reason about (has fewer state transitions)
- Picker
  - AbstractMovePicker - an abstract class with one method that takes a `Game` and returns a list of valid moves.
  - RandomMovePicker - asks the Game for valid moves and picks a random one from the list
  - MinimaxMovePicker - does the minimax algorithm on the given Game, can optionally prune
- play_game() - a function that takes a Game and an AbstractMovePicker and runs the game until completion, using the Picker to pick moves.


## Exercise 1
Expand the GameState class to include a method get_winner() that returns the winner of the game if the game is over and None otherwise. You will need to modify is_game_over() to keep track of who the winner is, or write get_winner() to not only detect a win but also identify the winner. 


In [28]:
def indent(depth):
    s = ""
    for _ in range(depth):
        s += ' '
    return s


from abc import ABC, abstractmethod

class Game(ABC):

    @abstractmethod
    def how_many_players(self) -> int:
        pass

    @abstractmethod
    def is_game_over(self) -> bool: 
        pass

    @abstractmethod
    def whose_turn(self) -> int:
        pass

    @abstractmethod
    def get_winner(self) -> int:
        pass

    @abstractmethod
    def get_valid_moves(self) -> list: 
        pass

    @abstractmethod
    def make_move(self, *args): 
        pass

    @abstractmethod
    def string(self, num_indent_spaces: int) -> str:
        pass

    def print(self, num_indent_spaces: int):
        print(self.string(num_indent_spaces))

    def indent(self, num_indent_spaces: int) -> str:
        return indent(num_indent_spaces)

class TicTacToe(Game):
    def __init__(self, num_players: int = 2, size: int = 3):
        # We represent the Tic-Tac-Toe board as a 3x3 array. 
        # We use 0 to represent an empty square, 1 for Player 1's piece, and 2 for Player 2's piece. 
        self.board = []
        for row in range(size):
            self.board.append([])
            for col in range(size):
                self.board[row].append(0)
        # Player 1 goes first. 
        self.player_to_move = 1 # random.choice(list(range(1, num_players+1)))
        self.num_players = num_players
        self.size = size
        # print(self)
        self.winner = 0
        self.done = False

    def how_many_players(self) -> int:
        return self.num_players

    def whose_turn(self) -> int:
        return self.player_to_move

    def is_game_over(self) -> bool: 
        return self.done

    def get_winner(self) -> int:
        return self.winner

    def get_valid_moves(self) -> list: 
        # Return a list of valid moves. 
        # A move is a tuple (row, col). 
        return [(row, col) for row in range(self.size) for col in range(self.size) if self.board[row][col] == 0] 

    def make_move(self, args): 
        row, col = args[0], args[1]
        # Make a move at the specified location. 
        # This assumes that the move is valid. 
        assert self.board[row][col] == 0 
        self.board[row][col] = self.player_to_move 
        # Switch the player to move. 
        self.player_to_move += 1
        if self.player_to_move > self.num_players:
            self.player_to_move = 1
        done, winner = self.get_status()
        self.winner = winner
        self.done = done

    def get_status(self):
        # Quick check
        for player in range(1, self.num_players + 1): 
            for row in range(self.size): 
                if all(self.board[row][col] == player for col in range(self.size)): 
                    return True, player 
            for col in range(self.size): 
                if all(self.board[row][col] == player for row in range(self.size)): 
                    return True, player 
            if all(self.board[i][i] == player for i in range(self.size)): 
                return True, player 
            if all(self.board[i][self.size-1-i] == player for i in range(self.size)): 
                return True, player 

        for row in range(self.size):
            for col in range(self.size):
                if self.board[row][col] == 0:
                    return False, 0 # Game not over yet
            
        return True, 0 # Cat's game

    def string(self, num_indent_spaces: int = 0) -> str:
        s = self.indent(num_indent_spaces)
        s += f'{self.player_to_move}/{self.num_players}\t Game Over: {self.done}\tWinner:{self.winner}'
        for row in range(self.size):
            s += '\n'
            s += self.indent(num_indent_spaces)
            s += str(self.board[row])
        return s



## Exercise 2 - random game play


In [61]:

from abc import ABC, abstractmethod

class AbstractMovePicker(ABC):
    @abstractmethod
    def pick_move(self, game: Game) -> any:
        pass    


import random

class RandomMovePicker(AbstractMovePicker):

    def pick_move(self, game):
        moves = game.get_valid_moves()
        return random.choice(moves)
        

def play_game(game: Game, move_picker: AbstractMovePicker, print_each_move = True):
    print('\nStarting Game!')
    while True:
        if game.is_game_over():
            if game.get_winner() == 0:
                print("It's a Draw!")
            else:
                print('Winner: ', game.get_winner())
            return
        
        mv = move_picker.pick_move(game)

        if print_each_move:
            print(f'*Player {game.whose_turn()} chose move {mv}: ')

        game.make_move(mv)

        if print_each_move:
            game.print(0)


play_game(TicTacToe(), RandomMovePicker(), print_each_move = True)




Starting Game!
*Player 1 chose move (1, 0): 
2/2	 Game Over: False	Winner:0
[0, 0, 0]
[1, 0, 0]
[0, 0, 0]
*Player 2 chose move (0, 0): 
1/2	 Game Over: False	Winner:0
[2, 0, 0]
[1, 0, 0]
[0, 0, 0]
*Player 1 chose move (1, 1): 
2/2	 Game Over: False	Winner:0
[2, 0, 0]
[1, 1, 0]
[0, 0, 0]
*Player 2 chose move (1, 2): 
1/2	 Game Over: False	Winner:0
[2, 0, 0]
[1, 1, 2]
[0, 0, 0]
*Player 1 chose move (2, 0): 
2/2	 Game Over: False	Winner:0
[2, 0, 0]
[1, 1, 2]
[1, 0, 0]
*Player 2 chose move (2, 2): 
1/2	 Game Over: False	Winner:0
[2, 0, 0]
[1, 1, 2]
[1, 0, 2]
*Player 1 chose move (0, 2): 
2/2	 Game Over: True	Winner:1
[2, 0, 1]
[1, 1, 2]
[1, 0, 2]
Winner:  1


### Exercise 3 - 3-player game

In [30]:
play_game(TicTacToe(num_players=3, size=4), RandomMovePicker(), print_each_move = True)



Starting Game!
*Player 1 chose move (3, 3): 
2/3	 Game Over: False	Winner:0
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
*Player 2 chose move (0, 2): 
3/3	 Game Over: False	Winner:0
[0, 0, 2, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
*Player 3 chose move (1, 2): 
1/3	 Game Over: False	Winner:0
[0, 0, 2, 0]
[0, 0, 3, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
*Player 1 chose move (3, 2): 
2/3	 Game Over: False	Winner:0
[0, 0, 2, 0]
[0, 0, 3, 0]
[0, 0, 0, 0]
[0, 0, 1, 1]
*Player 2 chose move (0, 1): 
3/3	 Game Over: False	Winner:0
[0, 2, 2, 0]
[0, 0, 3, 0]
[0, 0, 0, 0]
[0, 0, 1, 1]
*Player 3 chose move (2, 2): 
1/3	 Game Over: False	Winner:0
[0, 2, 2, 0]
[0, 0, 3, 0]
[0, 0, 3, 0]
[0, 0, 1, 1]
*Player 1 chose move (2, 3): 
2/3	 Game Over: False	Winner:0
[0, 2, 2, 0]
[0, 0, 3, 0]
[0, 0, 3, 1]
[0, 0, 1, 1]
*Player 2 chose move (1, 1): 
3/3	 Game Over: False	Winner:0
[0, 2, 2, 0]
[0, 2, 3, 0]
[0, 0, 3, 1]
[0, 0, 1, 1]
*Player 3 chose move (1, 0): 
1/3	 Game Over: False	Winner:0
[0, 2, 2, 0]
[3, 2,

# Exercises 2 - Minimax and Adversarial Game States

## Q1 - Copy()

In [31]:
# This one is simple
# i implementing copying simply using the copy.deepcopy function
game = TicTacToe()
game.make_move((1,1))

import copy
game2 = copy.deepcopy(game)

game.print(num_indent_spaces=0)
game2.print(num_indent_spaces=0)

2/2	 Game Over: False	Winner:0
[0, 0, 0]
[0, 1, 0]
[0, 0, 0]
2/2	 Game Over: False	Winner:0
[0, 0, 0]
[0, 1, 0]
[0, 0, 0]


# Exercises 3 - Minimax w/ Alpha/Beta Pruning

## Ex 1 - 5x5
You can see above that it already supports 5x5

In [32]:
game = TicTacToe(size=5)
game.make_move((1,1))
game.make_move((2,2))
game.print(num_indent_spaces=0)

1/2	 Game Over: False	Winner:0
[0, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 2, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]


## Q2
Write a function play_pruned_game() that plays a game using two AI players with the minimax algorithm and Alpha-Beta pruning. How does the performance compare with the version without pruning? Note that the 5x5 board is much more complex than the 3x3 board, so it’s not a direct comparison. 


Here I give my full minimax implementation.

You tell it whether to prune or not.

In [57]:
import copy as cp
import random

def evaluate(game: Game): 
    if game.get_winner() == 2:
        return -1
    if game.get_winner() == 1:
        return 1
    return 0

class MinimaxMovePicker(AbstractMovePicker): 
    def __init__(self, max_depth = 2, print_each_minimax = True, prune = True):
        if max_depth <= 0:
            raise ValueError('max depth has to be > 0')
        self.max_depth = max_depth
        self.print_each_minimax = print_each_minimax
        self.prune = prune
    
    def pick_move(self, game: Game):
        _, mv = self.minimax(game, 0, -float('inf'), float('inf'))
        return mv
        
    # returns best_score, best_move (or None)
    def minimax(self, game: Game, depth: int, alpha: int, beta: int):
        if game.how_many_players() > 2:
            raise ValueError(f"Can't do minimax with more than 2 players ({self.num_players})")

        if self.print_each_minimax:
            print(f'{indent(depth)}minimax state: ', game.string(depth))
            
        # If we've reached the maximum depth or the game is over, return the score. 
        if depth == self.max_depth or game.is_game_over(): 
            # print(f"{self.indent()} player {self.player_to_move} detects that the winner is player {self.winner}")
            val = evaluate(game)
            if self.print_each_minimax: # or val != 0:
                print(f"{indent(depth)}Eval: {val}")
            return val, None 

        # Initialize the best score and best move. 
        # Player 1 wants to maximize the score. 
        # Player 2 wants to minimize the score. 
        sign = 1 if game.whose_turn() == 1 else -1
        best_score = -float('inf') * sign # negative infinity for player 1, positive infinity for player 2
        best_moves = []

        # Try all possible moves. 
        for move in game.get_valid_moves():
            if self.print_each_minimax:
                print(f"{indent(depth)}evaluating move {move}") 
            # deep copy the game and make the move.
            next_game = cp.deepcopy(game)
            next_game.make_move(move) 
            score, _ = self.minimax(next_game, depth + 1, alpha, beta) 
            if self.print_each_minimax: # or score != 0:
                print(f"{indent(depth)}player {game.whose_turn()}: move {move} returned score {score}")

            # Update the best score and best move. 
            # TODO - randomly pick a move out of moves that are all equal?
            if sign * score == sign * best_score: 
                if self.print_each_minimax:
                    print(f"{indent(depth)}same move: {move} \t moves: {best_moves}")
                best_moves.append(move)
            if sign * score > sign * best_score: 
                if self.print_each_minimax:
                    print(f"{indent(depth)}better move: {move}")
                best_score = score 
                best_moves = [move]

            # Update the alpha/beta values 
            if game.whose_turn() == 1: 
                alpha = max(alpha, best_score) 
            else: 
                beta = min(beta, best_score) 
 
            # Alpha Beta Pruning condition 
            if beta <= alpha: 
                if self.print_each_minimax:
                    print(f"{indent(depth)}Pruning: {beta} <= {alpha}")
                if self.prune:
                    break 
                else:
                    if self.print_each_minimax:
                        print(f"{indent(depth)}NOT Pruning due to prune = False")

        return best_score, best_moves[random.randint(0, len(best_moves)-1)]


In [40]:
play_game(
    TicTacToe(size=5),
    MinimaxMovePicker(max_depth=2, print_each_minimax=False),
    print_each_move = True
)


Starting Game!
*Player 1 chose move (4, 4): 
2/2	 Game Over: False	Winner:0
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
*Player 2 chose move (0, 2): 
1/2	 Game Over: False	Winner:0
[0, 0, 2, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
*Player 1 chose move (1, 2): 
2/2	 Game Over: False	Winner:0
[0, 0, 2, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
*Player 2 chose move (1, 3): 
1/2	 Game Over: False	Winner:0
[0, 0, 2, 0, 0]
[0, 0, 1, 2, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
*Player 1 chose move (1, 4): 
2/2	 Game Over: False	Winner:0
[0, 0, 2, 0, 0]
[0, 0, 1, 2, 1]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
*Player 2 chose move (0, 1): 
1/2	 Game Over: False	Winner:0
[0, 2, 2, 0, 0]
[0, 0, 1, 2, 1]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
*Player 1 chose move (2, 2): 
2/2	 Game Over: False	Winner:0
[0, 2, 2, 0, 0]
[0, 0, 1, 2, 1]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 

### Results - 3x3
Here are the results of running 100 iterations at different max-depths on a 3x3:

In [51]:
results = [[1, 4, 69, 27],
[2, 28, 47, 25],
[3, 23, 45, 32],
[4, 29, 48, 23],
[5, 36, 50, 14],
[6, 42, 42, 16],
[7, 41, 41, 18],
[8, 35, 44, 21],
[9, 41, 45, 14],
[10, 42, 38, 20],
[11, 37, 44, 19],
[12, 41, 46, 13],
[13, 38, 44, 18],
]

import pandas as pd
df = pd.DataFrame(results, columns=['Depth', 'Draw', 'X wins', 'O wins'])

df

,Depth,Draw,X wins,O wins
0,1,4,69,27
1,2,28,47,25
2,3,23,45,32
3,4,29,48,23
4,5,36,50,14
5,6,42,42,16
6,7,41,41,18
7,8,35,44,21
8,9,41,45,14
9,10,42,38,20


As you can see, after about depth=5, the number of draws hovers around 40%, X a little higher, and O about 20% or less.

## Performance at 3x3
At higher depths, it slows down considerably.

In [55]:
# Code for iterations -- this will take 20s to run!

iterations = 10
# results = ['depth', 'draw', 'x', 'o']
results = []
for max_depth in range(1, 5):
    print('max_depth', max_depth)
    x = 0
    o = 0
    draw = 0
    for j in range(iterations):
        # print('iteration', j)
        game = TicTacToe(size=5) # size=5!!
        play_game(game, MinimaxMovePicker(max_depth = max_depth, print_each_minimax=False), print_each_move=False)
        if game.get_winner() == 0:
            draw += 1
        elif game.get_winner() == 1:   
            x += 1
        else:
            o += 1
    results.append([max_depth, draw, x, o])

df = pd.DataFrame(results, columns=['Depth', 'Draw', 'X wins', 'O wins'])
df

# below is results for size=5, depths 1-4, 10 iterations each:
# probably not conclusive, because it's such a small sample size
# but at depth=4, we get considerably slower.


max_depth 1
max_depth 2
max_depth 3
max_depth 4


,Depth,Draw,X wins,O wins
0,1,3,0,7
1,2,7,1,2
2,3,8,1,1
3,4,8,0,2


### Performance of Pruning
Performance is much better *with* pruning.


In [67]:

print('\n Try WITHOUT pruning -- it is slow')
play_game(
    TicTacToe(size=5),
    MinimaxMovePicker(max_depth=3, print_each_minimax=False, prune=False),
    print_each_move = False
)

print('\n Now try again WITH pruning')
play_game(
    TicTacToe(size=5),
    MinimaxMovePicker(max_depth=3, print_each_minimax=False, prune=True),
    print_each_move = False
)


 Try WITHOUT pruning -- it is slow

Starting Game!
It's a Draw!

 Now try again WITH pruning

Starting Game!
It's a Draw!


### Counting game
Here is my simpler game that is kind of like even/odds, that I used to test the minimax:

In [68]:
from game.game import Game

class Count(Game):
    def __init__(self, goal: int = 5, max_choice: int = 2):
        self.turn = 1
        self.goal = goal
        self.count = 0
        self.winner = None
        self.max_choice = max_choice

    def how_many_players(self) -> int:
        return 2

    def is_game_over(self) -> bool: 
        game_over = self.count >= self.goal
        # print(f'Count: {self.count} >= {self.goal} = {game_over}')
        return game_over

    def whose_turn(self) -> int:
        return self.turn

    def get_winner(self) -> int:
        return self.winner

    def get_valid_moves(self) -> list: 
        return range(1, self.max_choice + 1)

    def make_move(self, *args): 
        self.count += args[0]
        self.game_over = self.count >= self.goal
        if self.game_over:
            self.winner = self.turn
        else:
            self.turn = 3 - self.turn

    def string(self, num_indent_spaces: int) -> str:
        return self.indent(num_indent_spaces) + f'Count: {self.count}\tTurn: {self.turn} \t Winner:{self.winner}'



In [85]:
play_game(Count(goal=6), RandomMovePicker())
play_game(Count(goal = 6, max_choice=3), MinimaxMovePicker(max_depth = 5, print_each_minimax=False))



Starting Game!
*Player 1 chose move 1: 
Count: 1	Turn: 2 	 Winner:None
*Player 2 chose move 1: 
Count: 2	Turn: 1 	 Winner:None
*Player 1 chose move 2: 
Count: 4	Turn: 2 	 Winner:None
*Player 2 chose move 1: 
Count: 5	Turn: 1 	 Winner:None
*Player 1 chose move 1: 
Count: 6	Turn: 1 	 Winner:1
Winner:  1

Starting Game!
*Player 1 chose move 3: 
Count: 3	Turn: 2 	 Winner:None
*Player 2 chose move 3: 
Count: 6	Turn: 2 	 Winner:2
Winner:  2


# Exercises 4 - Monte Carlo Tree Search in Adversarial Game States

I didn't implement MCTS due to time constraints.

# Exercises 5

Note: I did work in Lucid and sometimes on a whiteboard.  These are the answers I got:

1. 19
2. 6, 21, and F and its children
3. 
